In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import utils as ut

show = 1
cur = ut.leer_csv("data/ARIA/5.CourseOfTreatment/_Course.csv", show).drop(columns=["ClinicalProtocolDir","Comment",
                 "CompletedByUserName","TransactionId","CourseId","CompletedDateTime"])

pi = ut.leer_csv("data/ARIA/5.CourseOfTreatment/_PhysicianIntent.csv", show).drop(columns=["CreationUserName","CreationDate",
                                                                                          "PhysicianIntentUID"])

# Diagnosis:
cd = ut.leer_csv("data/ARIA/5.CourseOfTreatment/CourseDiagnosis.csv", show)
diag_pri = ut.leer_csv("data/ARIA_merged&selected/Diagnosis_pri.csv", show).drop(columns="PatientSer")
diag_sec = ut.leer_csv("data/ARIA_merged&selected/Diagnosis_sec.csv", show).drop(columns="PatientSer")

a1 = ut.leer_csv("data/ARIA/7.ActivityAndBilling/_ActCaptDiagnosis.csv", show)
a2 = ut.leer_csv("data/ARIA/7.ActivityAndBilling/_ActivityCapture.csv", show)

Archivo: data/ARIA/5.CourseOfTreatment/_Course.csv
Se dejaron 10 columnas.
Se eliminaron 4 columnas:


Archivo: data/ARIA/5.CourseOfTreatment/_PhysicianIntent.csv
Se dejaron 6 columnas.
Se eliminaron 5 columnas:


Archivo: data/ARIA/5.CourseOfTreatment/CourseDiagnosis.csv
Se dejaron 3 columnas.
Se eliminaron 4 columnas:


Archivo: data/ARIA_merged&selected/Diagnosis_pri.csv
Se dejaron 10 columnas.
Se eliminaron 0 columnas:


Archivo: data/ARIA_merged&selected/Diagnosis_sec.csv
Se dejaron 5 columnas.
Se eliminaron 0 columnas:


Archivo: data/ARIA/7.ActivityAndBilling/_ActCaptDiagnosis.csv
Se dejaron 8 columnas.
Se eliminaron 7 columnas:


Archivo: data/ARIA/7.ActivityAndBilling/_ActivityCapture.csv
Se dejaron 10 columnas.
Se eliminaron 12 columnas:




In [2]:
cur_pi = cur.merge(pi, on="CourseSer", how="outer", suffixes = ("_cur", "_pi")).drop(columns=["PhysicianIntentSer"])
cur_pi

,PatientSer,ClinicalStatus,StartDateTime,CourseSer,TreatmentIntentType
0,4,ACTIVE,1/14/2016 9:47:24 AM,11,NaN
1,9,ACTIVE,NaN,32,NaN
2,7,ACTIVE,1/15/2016 10:03:30 AM,36,NaN
3,8,ACTIVE,1/15/2016 10:13:15 AM,37,NaN
4,11,ACTIVE,NaN,51,NaN
...,...,...,...,...,...
6425,7026,ACTIVE,8/7/2020 12:00:00 AM,10416,NaN
6426,7028,ACTIVE,8/7/2020 4:28:37 PM,10421,NaN
6427,7026,ACTIVE,8/10/2020 9:40:32 AM,10431,NaN
6428,19,ACTIVE,9/9/2020 8:42:13 AM,10441,NaN


# Relacción Course-Diagnosis
Recordar: DiagnosisSer puede referirse a una primaria o secundaria

In [3]:
cd2 = a2[["ActivityCaptureSer","CourseSer","ActivityInstanceSer"]] \
.merge(a1[["DiagnosisSer","ActivityCaptureSer"]], on="ActivityCaptureSer", how="outer") \
[["CourseSer","DiagnosisSer"]].drop_duplicates().dropna()

cdf = pd.concat([cd, cd2])[["CourseSer","DiagnosisSer"]].drop_duplicates()

In [4]:
# me quedo con aquellas relaciones Course-Diagnosis donde el curso solo tiene asociada una diagnosis
vc = cdf[cdf.DiagnosisSer.isin(diag_pri.DiagnosisSer)].CourseSer.value_counts()
c1 = vc[vc==1].index

vc = cdf[cdf.DiagnosisSer.isin(diag_sec.DiagnosisSer)].CourseSer.value_counts()
c2 = vc[vc==1].index

cd_pri = cdf[(cdf.CourseSer.isin(c1)) & cdf.DiagnosisSer.isin(diag_pri.DiagnosisSer)]
cd_sec = cdf[(cdf.CourseSer.isin(c2)) & cdf.DiagnosisSer.isin(diag_sec.DiagnosisSer)]

In [5]:
print(cd_pri.shape)
print(cd_pri.CourseSer.nunique())
print(cd_pri.DiagnosisSer.nunique())

ut.rel_indices(cd_sec.CourseSer, cur_pi.CourseSer)

(4589, 2)
4589
4305
Longitud del primer set: 663
Longitud del segundo set: 6430

Indices comunes: 663
Indices que solo están en el primero: 0
Indices que solo están en el segundo: 5767
_____________________________________________



In [6]:
cur_pi_diag = cur_pi.merge(cd_pri, on="CourseSer", how="left", suffixes = ("_curpi", "_cd_pri"))\
.merge(diag_pri, on="DiagnosisSer", how="left", suffixes = ("_curpi", "_diag_pri")).drop(columns="DiagnosisSer")\
.merge(cd_sec, on="CourseSer", how="left", suffixes = ("_curpi", "_cd_sec"))\
.merge(diag_sec, on="DiagnosisSer", how="left", suffixes = ("_curpi", "_diag_sec")).drop(columns="DiagnosisSer")

In [7]:
cur_pi_diag
ut.resumen_tabla(cur_pi_diag, "nans")

6430 registros
16 variables


,nans,nunicos,nunicos_r,dtype
PatientSer,0.00,5413,84.18,int64
ClinicalStatus,0.00,2,0.03,object
CourseSer,0.00,6430,100.00,int64
StartDateTime,3.00,6236,96.98,object
TreatmentIntentType,7.84,15,0.23,object
DiagnosisCode,28.63,422,6.56,object
ObjectStatus,28.63,2,0.03,object
SummaryStage,42.75,26,0.40,object
TStage,49.61,80,1.24,object
MStage,51.57,13,0.20,object


In [8]:
cur_pi_diag.to_csv("data/ARIA_merged&selected/Course_pi_diag.csv", index=False)